# Loan Data Analysis

In this analysis, we will be exploring the LendingClub.com dataset that contains data from almost 10,000 borrowers. We will be analyzing the most common types of loans and how long borrowers take to pay their investors. Finally, we will explore the relationship between the borrower's profile on their loan application and if it is fully paid. With this analysis, we hope to help lenders to make data-driven decisions on their investments by pre-determining features of borrowers who fully paid their loans had.

## 1\. Exploratory Data Analysis

To start, let's take a look at the first five rows of our dataset. This dataset has been cleaned already in our previous step, and you can refer to the Data Dictionary for more information.

In [1]:
--Selecting the top 5 rows
SELECT TOP(5) *
FROM [Projects].[dbo].[loan_data]

(5 rows affected)

Total execution time: 00:00:00.027

credit_policy,purpose,int_rate,installment,log_annual_inc,dti,fico,days_with_cr_line,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,pub_rec,not_fully_paid
1,Debt consolidation,0.1189,829.1,11.35040654,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,Credit card,0.1071,228.22,11.08214255,14.29,707,2760,33623,76.7,0,0,0,0
1,Debt consolidation,0.1357,366.86,10.37349118,11.63,682,4710,3511,25.6,1,0,0,0
1,Debt consolidation,0.1008,162.34,11.35040654,8.1,712,2699.958333,33667,73.2,1,0,0,0
1,Credit card,0.1426,102.92,11.29973224,14.97,667,4066,4740,39.5,0,1,0,0


Now that we know our dataset content, we will first explore our data and determine the following:

- How many loans were taken for each category? 
- How many loans were fully paid?
- If the borrower meets the credit policy, are they more likely to pay the loan?

In [2]:
-- How many loans were taken for each category?

SELECT purpose, COUNT(*) as loans
FROM [dbo].[loan_data] 
GROUP BY purpose
ORDER BY loans DESC

(7 rows affected)

Total execution time: 00:00:00.018

purpose,loans
Debt consolidation,3957
All other,2331
Credit card,1262
Home improvement,629
Small business,619
Major purchase,437
Educational,343


In [4]:
-- How many loans were fully paid?
SELECT 
CASE 
	WHEN not_fully_paid = 0 THEN 'Yes'
	ELSE 'No'
	END as status, COUNT(*) as loans
FROM [dbo].[loan_data] 
GROUP BY not_fully_paid

(2 rows affected)

Total execution time: 00:00:00.021

status,loans
Yes,8045
No,1533


In [5]:
-- If the borrower meets the credit policy, are they more likely to pay the loan?
SELECT CASE 
	WHEN credit_policy = 1 THEN 'Meets Criteria'
	ELSE 'Do not meet criteria'
	END as meet_criteria, 
CASE 
	WHEN not_fully_paid = 0 THEN 'Yes'
	ELSE 'No'
	END as fully_paid, COUNT(*) as loans
FROM [dbo].[loan_data] 
GROUP BY not_fully_paid, credit_policy
ORDER BY loans DESC

(4 rows affected)

Total execution time: 00:00:00.016

meet_criteria,fully_paid,loans
Meets Criteria,Yes,6696
Do not meet criteria,Yes,1349
Meets Criteria,No,1014
Do not meet criteria,No,519


<span style="background: transparent; margin-top: 0pt; margin-bottom: 0pt;" data-preserver-spaces="true">It seems that the most common type of loan is Debt Consolidation, Others, and Credit Card. We also found that from 9,578 loans, 8045 were fully paid, and most of these loans met the credit criteria required by LendingClub.com.&nbsp;</span> 

## <span style="background: transparent; margin-top: 0pt; margin-bottom: 0pt;" data-preserver-spaces="true">2. Evaluating a Low-risk Profile.</span>

<span style="background: transparent; margin-top: 0pt; margin-bottom: 0pt;" data-preserver-spaces="true">We now have an initial idea of what type of loans the borrowers usually take and their payment status. Since more than half of our borrowers paid their loans, we can investigate what features they have to determine a profile of a low-risk borrower. This will help investors to decide rather or not to invest according to other attributes besides the credit criteria of LendingClub.com. In order to evaluate what a low-risk profile looks like, let's analyze the following relationships:&nbsp;</span> 

- <span style="background: transparent; margin-top: 0pt; margin-bottom: 0pt;" data-preserver-spaces="true">The relationship between FICO credit score and payment status.</span>
- <span style="background: transparent; margin-top: 0pt; margin-bottom: 0pt;" data-preserver-spaces="true">The relationship between annual income and payment status.</span>
- <span style="background: transparent; margin-top: 0pt; margin-bottom: 0pt;" data-preserver-spaces="true">The relationship between derogatory public records and payment status.&nbsp;</span>

In [1]:
-- Relationship between FICO and payment status
WITH fico_score AS (SELECT CASE 
	WHEN fico < 650  THEN 'FICO less 650'
	WHEN fico >= 650 AND fico < 750 THEN 'FICO 650 - 750'
	WHEN fico >= 750 and fico < 800 THEN 'FICO 750 - 799'
	ELSE 'FICO above 800'
	END as FICO_score,
	CASE 
	WHEN not_fully_paid = 0 THEN 'Yes'
	ELSE 'No'
	END as fully_paid
	FROM [dbo].[loan_data] )
SELECT  FICO_score, fully_paid, COUNT(*) as loans
FROM fico_score
GROUP BY  FICO_score, fully_paid
ORDER BY loans DESC

(8 rows affected)

Total execution time: 00:00:01.049

FICO_score,fully_paid,loans
FICO 650 - 750,Yes,6342
FICO 750 - 799,Yes,1411
FICO 650 - 750,No,1335
FICO less 650,Yes,157
FICO above 800,Yes,135
FICO 750 - 799,No,114
FICO less 650,No,74
FICO above 800,No,10


In [2]:
-- Relationship between annual income and payment status
-- Since the annual income had a log transformation, I had to reverse it to normal number and rounded to 3 decimals
-- To simplify the analysis, I grouped the income into ranges.
WITH annual_inc AS (
SELECT  
	CASE 
		WHEN ROUND(EXP(log_annual_inc),3) < 50000 THEN 'Less than 50000'
		WHEN ROUND(EXP(log_annual_inc),3) >= 50000 AND ROUND(EXP(log_annual_inc),3)  < 90000 THEN '50000 - 89999'
		WHEN ROUND(EXP(log_annual_inc),3) >= 90000 AND ROUND(EXP(log_annual_inc),3) < 150000 THEN '90000 - 149999'
		ELSE 'More than 150000'
	END as annual_inc,
	CASE 
		WHEN not_fully_paid = 0 THEN 'Yes'
		ELSE 'No'
		END as fully_paid
FROM [dbo].[loan_data] )
SELECT annual_inc, fully_paid,  COUNT(*) as loans
FROM annual_inc
GROUP BY annual_inc, fully_paid
ORDER BY loans DESC

(8 rows affected)

Total execution time: 00:00:00.031

annual_inc,fully_paid,loans
Less than 50000,Yes,3201
50000 - 89999,Yes,3192
90000 - 149999,Yes,1241
Less than 50000,No,679
50000 - 89999,No,568
More than 150000,Yes,411
90000 - 149999,No,188
More than 150000,No,98


In [4]:
--Relationship between derogatory public records and payment status
SELECT 
	CASE 
		WHEN not_fully_paid = 0 THEN 'Yes'
		ELSE 'No'
		END as fully_paid, pub_rec as num_pub_rec, COUNT(*) as loans
FROM [dbo].[loan_data] 
WHERE pub_rec > 0
GROUP BY not_fully_paid, pub_rec
ORDER BY pub_rec 

(7 rows affected)

Total execution time: 00:00:00.024

fully_paid,num_pub_rec,loans
Yes,1,397
No,1,136
Yes,2,17
No,2,2
Yes,3,5
Yes,4,1
Yes,5,1


After exploring some relationships between the characteristics of the borrowers and their payment status, we can determine how a low-risk borrower profile would be. The first relationship we analyzed was the FICO score and payment status. From our query, we notice that the majority of the borrowers who fully paid their loans have a FICO score between 650 - 750. We also note a high number (1335) of not-fully-paid loans for FICO scores above 650 and only a few for scores less than 650. This result emphasizes that only the FICO score is not a strong indicator that the borrower is more likely to pay their loans.  The second relationship we analyzed was annual income and payment status. It does not seem that the annual income has a major influence on borrowers who paid their loans. But, we found that the majority of people who did not pay their loans have an annual income of less than 90,000. Finally, in the relationship between payment status and derogatory public records, we found that most people who have one or more derogatory records still paid their loans. Therefore, derogatory public records should not be a strong sign that the borrower will not pay the loan. 

### Low-Risk Borrower Profile:

According to our analysis, a low-risk profile would have:

- FICO score above 650.
- Annual income above 50,000.
- One or no derogatory public records.

## 3\. How long does it take for a loan to be fully paid?

Since our database does not have much information on when the loan was taken and when it was paid, we will be using the days with the credit line as a time measure. To find the average years that it takes for a loan to be fully paid, we will divide the column by number of days per year and take the average.

In [5]:
--Here we divided the day_with_cr_line column by 365 to find number of years
--Then take the average
--And round up to 2 decimals place.
SELECT  ROUND((AVG(days_with_cr_line/ 365)) , 2) AS avg_years
FROM [dbo].[loan_data] 
--Display results for only where the fully paid is true
WHERE not_fully_paid = 0

(1 row affected)

Total execution time: 00:00:00.010

avg_years
12.58


In [7]:
--Checking average years for each category
SELECT  purpose, ROUND((AVG(days_with_cr_line/ 365)) , 2) AS avg_years
FROM [dbo].[loan_data] 
WHERE not_fully_paid = 0
GROUP BY purpose
ORDER BY avg_years DESC

(7 rows affected)

Total execution time: 00:00:00.026

purpose,avg_years
Home improvement,14.36
Small business,13.44
Credit card,13.28
Debt consolidation,12.49
Major purchase,12
All other,11.99
Educational,11.17


From our queries, it seems that the average time for loans to be paid in 12 years. And the Home improvement has the longest average time compared to other categories.

# Conclusion

For this analysis, we used SQL to query the Loan Database to help lenders make a data-driven decision on their investments. We used the dataset to analyze the borrowers who fully paid their loans to establish a low-risk profile. We found some attributes such as FICO score or annual income do not have a strong influence on whether the loan would be paid or not. And the majority of people with derogatory public records still paid their loans. After this analysis, we could determine the features of a low-risk profile. Finally, we used the number of days credit line to estimate the average time a borrower takes to pay their loan. We also analyzed the average time for each category and concluded that the Home Improvement category has the higher average time for being fully paid.